# Data Preperation With Web Scraping 

In [9]:
from bs4 import BeautifulSoup  
import pandas as pd
from tqdm import tqdm
import math
import requests  
import lxml
import re
import time
import numpy as np

In [85]:
# if regular expression fails, return np.nan
def re_match(re_pattern, string, pattern2 = None, errif=np.nan):
    try:
        return re.findall(re_pattern, string)[0].strip()
    except IndexError:
        if pattern2 != None:
            return re.findall(pattern2, string)[0].strip()
        return errif

In [129]:
def scrap(citycode, area_dic):
    '''
    Input citycode and a dictionary of areas to scrape from Lian Jia website 
    '''
    data = pd.DataFrame()

    #header
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/65.0.3325.146 Safari/537.36',
               'Referer': f'https://{citycode}.lianjia.com/ershoufang/'}
    # create a new session
    sess = requests.session()
    sess.get(f'https://{citycode}.lianjia.com/ershoufang/', headers=headers)
    # ex. https://sz.lianjia.com/ershoufang/luohuqu/pg2/
    url = f'https://{citycode}'+'.lianjia.com/ershoufang/{}/pg{}/'

    for key_, value_ in area_dic.items():
        # total number of real estate
        start_url = 'https://{}.lianjia.com/ershoufang/{}/'.format(citycode, value_)
        html = sess.get(start_url).text
        house_num = re.findall('共找到<span> (.*?) </span>套.*二手房', html)[0]
        print('👻 District: {}\n😈 Total real estate for sale:「{}」'.format(key_, house_num))
        time.sleep(1)
        # website constraint
        total_page = int(math.ceil(min(3000, int(house_num)) / 30.0))
        for i in tqdm(range(total_page), colour = 'WHITE', desc=key_):
            html = sess.get(url.format(value_, i+1)).text
            soup = BeautifulSoup(html, 'lxml')
            info_collect = soup.find_all(class_="info clear")

            for info in info_collect:
                info_dic = {}
                # district
                info_dic['area'] = key_
                # title
                info_dic['title'] = re_match('target="_blank">(.*?)</a><!--', str(info))
                # community name
                info_dic['community'] = re_match('xiaoqu.*?target="_blank">(.*?)</a>', str(info))
                # location
                info_dic['position'] = re_match('<a href.*?target="_blank">(.*?)</a>.*?class="address">', str(info))
                # unit price
                info_dic['unit_price'] = str(re_match('<div class="unitPrice".*?><span>(.*?)元/平</span></div>', str(info)))

                # tax related, deeds over 5 years 
                info_dic['tax'] = re_match('class="taxfree">(.*?)</span>', str(info))
                # star received/ release time 
                info_dic['star'] = re_match('class="starIcon"></span>(.*?)人关注.*?</div>', str(info))
                info_dic['release_time'] = re_match('class="starIcon"></span>.*? / (.*?)以前发布</div>', str(info))
                # whether close to subway stations
                info_dic['subway'] = re_match('class="subway">(.*?)</span>', str(info))
                # allows VR inspection
                info_dic['vr'] = re_match('class="vr">(.*?)</span>', str(info))
                # has key 
                info_dic['has_key'] = re_match('class="haskey">(.*?)</span>', str(info)) 


                # other tages, splited by |
                icons = re.findall('class="houseIcon"></span>(.*?)</div>', str(info))[0].strip().split('|')
                info_dic['house_type'] = icons[0].strip()
                if len(icons)>1:
                    info_dic['house_size'] = float(icons[1].replace('平米', ''))
                if len(icons)>2:
                    info_dic['direction'] = icons[2].strip()
                if len(icons)>3:
                    info_dic['fitment'] = icons[3].strip()
                if len(icons)>4:
                    info_dic['floor'] = icons[4].strip()
                if len(icons)>5:
                    info_dic['year_built'] = icons[5].strip()
                if len(icons)>6:
                    info_dic['structure_type'] = icons[6].strip()
                if len(icons)>7:
                    info_dic['other_feature'] = icons[7].strip()

                # saved to dataframe
                if data.empty:
                    data = pd.DataFrame(info_dic,index=[0])
                else:
                    data = data.append(info_dic,ignore_index=True)
    return data 

In [144]:
#All districts: 锦江 青羊 武侯 高新 成华 金牛 天府新区 高新西 双流 温江 郫都 龙泉驿 新都 青白江
jinjiang = {'川师':'chuanshi','大面':'damian','东大路':'dongdalu','东光小区':'dongguangxiaoqu','东湖':'donghu',
'东客站':'dongkezhan','合江亭':'hejiangting','红星路':'hongxinglu','静居寺':'jingjusi','九眼桥':'jiuyanqiao',
'蓝谷地':'langudi','莲花':'lianhua2','琉璃场':'liulichang','攀成钢':'panchenggang','三官堂':'sanguantang',
'三圣乡':'sanshengxiang','沙河堡':'shahebao','水碾河':'shuinianhe','盐市口':'yanshikou','卓锦城':'zhuojincheng'}

qingyang = {'八宝街':'babaojie','贝森':'beisen','草市街':'caoshijie','草堂':'caotang','府南新区':'funanxinqu',
'光华泡小':'guanghuapaoxiao','浣花溪':'huanhuaxi','金沙':'jinsha','宽窄巷子':'kuanzhaixiangzi','人民公园':'renmingongyuan',
'蜀汉路':'shuhanlu','太升路':'taishenglu','外光华':'waiguanghua','外金沙':'waijinsha','万家湾':'wanjiawan',
'西南财大':'xinancaida','盐市口':'yanshikou','优品道':'youpindao'}

wuhou = {'草金立交':'caojinlijiao','川大':'chuanda','川音':'chuanyin','簇桥':'cuqiao','高升桥':'gaoshengqiao',
'广福桥':'guangfuqiao','航空路':'hangkonglu','红牌楼':'hongpailou','华西':'huaxi','火车南站':'huochenanzhan',
'丽都':'lidu','龙湾':'longwan','双楠':'shuangnan','桐梓林':'tongzilin','外双楠':'waishuangnan','五大花园':'wudahuayuan',
'武侯祠':'wuhouci','武侯立交':'wuhoulijiao','新双楠':'xinshuangnan','玉林':'yulin','棕北':'zongbei'}

gaoxin = {'城南宜家':'chengnanyijia','大源':'dayuan','东苑':'dongyuan2','芳草':'fangcao','高朋':'gaopeng','广都':'guangdou',
'华府':'huafu1','华阳':'huayang','金融城':'jinrongcheng','丽都':'lidu','神仙树':'shenxianshu','市一医院':'shiyiyiyuan',
'天府长城':'tianfuchangcheng','新北':'xinbei','新会展':'xinhuizhan','新双楠':'xinshuangnan','衣冠庙':'yiguanmiao',
'远大':'yuanda','中德':'zhongde','中和':'zhonghe','紫荆':'zijing'}

chenghua = {'八里小区':'balixiaoqu','成渝立交':'chengyulijiao','东郊记忆':'dongjiaojiyi','东客站':'dongkezhan','动物园':'dongwuyuan',
'建设路':'jianshelu','理工大':'ligongda','李家沱':'lijiatuo','龙潭寺':'longtansi','猛追湾':'mengzhuiwan','沙河堡':'shahebao',
'驷马桥':'simaqiao','SM广场':'smguangchang','万年场':'wannianchang','万象城':'wanxiangcheng1','新华公园':'xinhuagongyuan'}

jinniu = {'八宝街':'babaojie','茶店子':'chadianzi','抚琴小区':'fuqinxiaoqu','高家庄':'gaojiazhuang','国宾':'guobin',
'花牌坊':'huapaifang','华侨城':'huaqiaocheng3','花照壁':'huazhaobi','金府':'jinfu','金牛万达':'jinniuwanda',
'九里堤':'jiulidi','马鞍路':'maanlu','沙湾':'shawanhuizhan','石人小区':'shirenxiaoqu','蜀汉路':'shuhanlu','天回镇':'tianhuizhen',
'通惠门':'tonghuimen','五块石':'wukuaishi','西南交大':'xinanjiaoda','营门口':'yingmenkou','一品天下':'yipintianxia'}

tianfuxinqu = {'海洋公园':'haiyanggongyuan','华阳':'huayang','锦江生态带':'jinjiangshengtaidai','麓湖生态城':'luhushengtaicheng',
'麓山':'lushan','南湖':'nanhu','仁寿':'renshou1','四河':'sihe','新会展':'xinhuizhan','雅居乐':'yajule'}

gaoxinxi = {'高新西':'gaoxinxi','中海国际':'zhonghaiguoji'}

shuangliu = {'簇桥':'cuqiao','东升镇':'dongshengzhen','公兴':'gongxing','航空港':'hangkonggang','蛟龙港':'jiaolonggang',
'警院':'jingyuan','九龙湖':'jiulonghu1','牧马山':'mumashan','双流城区':'shuangliuchengqu','文星镇':'wenxingzhen',
'新津':'xinjin'}

wenjiang = {'芙蓉古城':'furonggucheng','光华大道沿线':'guanghuadadaoyanxian','国色天乡':'guosetianxiang','花都大道':'huadudadao',
'温江大学城':'wenjiangdaxuecheng','温江老城':'wenjianglaocheng','温江新城':'wenjiangxincheng','珠江新城':'zhujiangxincheng'}

pidu = {'成外':'chengwai','红光':'hongguang','郫县城区':'pixianchengqu','郫县万达':'pixianwanda','橡树湾':'xiangshuwan',
'犀浦':'xipu'}

longquanyi = {'东山':'dongshan','航天':'hangtian','洪河':'honghe','龙泉驿城区':'longquanyichengqu','十陵':'shiling2',
'西河':'xihe','阳光城':'yangguangcheng'}

xindu = {'保利公园':'baoligongyuan','大丰':'dafeng','毗河':'pihe','新都城区':'xinduchengqu'}

qingbaijiang = {'青白江':'qingbaijiang'}


inner_districts = [jinjiang, qingyang, wuhou, jinniu, gaoxin, chenghua]
outer_districts = [tianfuxinqu, gaoxinxi, shuangliu, wenjiang, pidu, longquanyi, xindu, qingbaijiang]

In [131]:
inner_data = pd.DataFrame()
outer_data = pd.DataFrame()
cd_data = pd.DataFrame()
for district in inner_districts:
    district_data = scrap('cd',district)
    inner_data = pd.concat([inner_data, district_data])

👻 District: 川师
😈 Total real estate for sale:「375」


川师: 100%|██████████| 13/13 [00:29<00:00,  2.28s/it]


👻 District: 大面
😈 Total real estate for sale:「2378」


大面: 100%|██████████| 80/80 [02:44<00:00,  2.05s/it]


👻 District: 东大路
😈 Total real estate for sale:「257」


...

万年场: 100%|██████████| 43/43 [01:51<00:00,  2.58s/it]


👻 District: 万象城
😈 Total real estate for sale:「930」


万象城: 100%|██████████| 31/31 [01:21<00:00,  2.64s/it]


👻 District: 新华公园
😈 Total real estate for sale:「777」


新华公园: 100%|██████████| 26/26 [01:12<00:00,  2.78s/it]


In [132]:
for district in outer_districts:
    district_data = scrap('cd',district)
    outer_data = pd.concat([outer_data, district_data])

👻 District: 海洋公园
😈 Total real estate for sale:「770」


海洋公园: 100%|██████████| 26/26 [01:01<00:00,  2.36s/it]


👻 District: 华阳
😈 Total real estate for sale:「1807」


华阳: 100%|██████████| 61/61 [02:31<00:00,  2.48s/it]


👻 District: 锦江生态带
😈 Total real estate for sale:「588」


...

毗河: 100%|██████████| 44/44 [01:50<00:00,  2.51s/it]


👻 District: 新都城区
😈 Total real estate for sale:「2784」


新都城区: 100%|██████████| 93/93 [03:51<00:00,  2.49s/it]


👻 District: 青白江
😈 Total real estate for sale:「2687」


青白江: 100%|██████████| 90/90 [03:46<00:00,  2.52s/it]


In [139]:
cd_data = pd.concat([outer_data,inner_data],axis = 0)

In [140]:
cd_data.to_csv('cd_raw.csv', index = False)

In [141]:
inner_data.to_csv('cd_inner_raw.csv', index = False)
outer_data.to_csv('cd_outer_raw.csv', index = False)